# Multi-Layer Perceptron (MLP) Implementation with C-Wrapper

This notebook contains the implementation of an **MLP** class designed to interface with a custom C-wrapper (`c_wrapper`) for efficient, low-level matrix operations. It handles core neural network functions, including initialization, forward propagation, backpropagation, and training.

The internal structure assumes samples are represented as columns (features $\times$ samples), necessitating transpositions when interacting with external NumPy data (samples $\times$ features).

In [ ]:
import c_wrapper
import numpy as np
from typing import List, Optional

In [ ]:
class MLP:    
    def __init__(self, layer_sizes: List[int], hidden_activation: str, output_activation: str, loss: str = "MSE", learning_rate=0.01, seed: Optional[int] = None):
        self.layer_sizes = layer_sizes
        self.hidden_activation = hidden_activation.lower()
        self.output_activation = output_activation.lower()
        self.loss = loss
        self.learning_rate = learning_rate
        self.activations = []
        self.weights = []
        self.biases = []

        if seed is not None:
            np.random.seed(seed)

        for i in range(len(self.layer_sizes) - 1):
            limit = np.sqrt(2 / layer_sizes[i])
            w_np = np.random.randn(layer_sizes[i + 1], layer_sizes[i]) * limit
            b_np = np.zeros((layer_sizes[i + 1], 1))

            self.weights.append(c_wrapper.from_numpy(w_np))    
            self.biases.append(c_wrapper.from_numpy(b_np))


## `_clear_activations`: Memory Management

This is a crucial helper function responsible for **explicit memory cleanup**. It iterates through the stored layer activations and calls `c_wrapper.free_py_matrix` to release the associated C-level memory after the backward pass, preventing memory leaks during training.

In [ ]:
def _clear_activations(self):
    for i in range(1, len(self.activations)):
        c_wrapper.free_py_matrix(self.activations[i])
    self.activations = []


## `forward_pass`: Feedforward Calculation

This method computes the network's output. For each layer, it performs the linear transformation $Z = W \cdot A_{\text{prev}} + B$ and applies the specified **activation function** (`sigmoid`, `relu`, or `identity`) to get $A = g(Z)$. This involves matrix multiplication of the current weights with the previous layer's output, followed by adding the bias vector and running the result through the non-linear function. All intermediate activations $A$ are stored in self.activations for use during backpropagation.

In [ ]:
def forward_pass(self, X: c_wrapper.Matrix):
    self._clear_activations()

    self.activations = [X]
    cur_output = X
    num_layers = len(self.weights)

    for layer_index in range(num_layers):
        weights = self.weights[layer_index]
        biases = self.biases[layer_index]

        weights_matrix = c_wrapper.multiply_py_matrices(
            weights, cur_output)
        input_matrix = c_wrapper.py_add_weights_and_biases(
            weights_matrix, biases)

        c_wrapper.free_py_matrix(weights_matrix)

        is_output_layer = (layer_index == num_layers - 1)
        activation_type = self.output_activation if is_output_layer else self.hidden_activation

        if activation_type == "sigmoid":
            activated_matrix = c_wrapper.py_sigmoid(input_matrix)
            c_wrapper.free_py_matrix(input_matrix)
        elif activation_type == "relu":
            activated_matrix = c_wrapper.py_matrix_relu(input_matrix)
            c_wrapper.free_py_matrix(input_matrix)
        elif activation_type == "identity":
            activated_matrix = input_matrix
        else:
            raise ValueError(
                f"The activation function {activation_type} has not been implemented"
            )

        cur_output = activated_matrix
        self.activations.append(cur_output)

    return cur_output


## `backward_pass`: Backpropagation and Weight Update

This method implements the **backpropagation algorithm**, which updates the network’s weights and biases by propagating errors backward through the layers. It begins by computing the gradient of the Mean Squared Error (MSE) loss and then applies the chain rule layer by layer.

### Loss Gradient
For MSE loss:
$$
\nabla_{\text{Loss}} = \hat{y} - y
$$

### Output Layer
The error signal (delta) at the output layer is computed as:
$$
\delta_L = \nabla_{\text{Loss}} \odot g'(Z_L)
$$
where $g'(Z_L)$ is the derivative of the output activation function.

### Hidden Layers
For hidden layers, the error signal propagates backward using:
$$
\delta_l = (W_{l+1}^T \cdot \delta_{l+1}) \odot g'(Z_l)
$$
This multiplies the next layer’s delta by the transposed weight matrix and then element-wise by the derivative of the current layer’s activation.

### Gradient Computation
Once all deltas are known, the gradients for the weights and biases are computed:
$$
\nabla W_l = \delta_l \cdot A_{l-1}^T, \quad
\nabla B_l = \delta_l
$$
where $A_{l-1}$ is the activation from the previous layer.

### Parameter Update
Weights and biases are then updated using gradient descent:
$$
W_l = W_l - \eta \nabla W_l, \quad
B_l = B_l - \eta \nabla B_l
$$
Here, $\eta$ is the learning rate controlling the step size, balancing convergence speed and stability.

### Memory Management
Throughout the backpropagation process, numerous temporary matrices are created to store deltas, transposes, gradients, and activation derivatives. The implementation explicitly frees these matrices as soon as they are no longer needed via `free_py_matrix`, ensuring efficient memory use and preventing leaks in environments without automatic garbage collection.


In [ ]:
def backward_pass(self, y_true: c_wrapper.Matrix,
                    y_pred: c_wrapper.Matrix):
    if self.loss != "MSE":
        raise ValueError(f"{self.loss} has not been defined yet.")

    # Gradient for MSE: 2 * (y_pred - y_true) / N
    diff = c_wrapper.subtract_py_matrices(y_pred, y_true)
    output_error = c_wrapper.scalar_multiply_py_matrix(
        diff, 2 / (y_true.cols * y_true.rows))
    c_wrapper.free_py_matrix(diff)

    num_layers = len(self.weights)

    for layer_index in range(num_layers - 1, -1, -1):

        is_output_layer = (layer_index == num_layers - 1)
        activation_type = self.output_activation if is_output_layer else self.hidden_activation

        activation_derivative = None
        if activation_type == "sigmoid":
            activation_derivative = c_wrapper.py_sigmoid_derivative(
                self.activations[layer_index + 1])
        elif activation_type == "relu":
            activation_derivative = c_wrapper.py_matrix_relu_derivative(
                self.activations[layer_index + 1])
        elif activation_type == "identity":
            # For identity, derivative is 1. We handle the error signal below.
            pass
        else:
            raise ValueError(
                f"Unsupported activation {activation_type} in backward_pass"
            )

        # Compute error signal (delta)
        if layer_index == num_layers - 1:
            # Output Layer: Loss_grad * Act_Derivative
            if activation_type == "identity":
                error_signal = c_wrapper.scalar_multiply_py_matrix(
                    output_error, 1)
            else:
                error_signal = c_wrapper.hadamard_py_matrices(
                    output_error, activation_derivative)
        else:
            # Hidden Layer: (W_T * Previous_Error) * Act_Derivative
            transposed_weights = c_wrapper.transpose_py_matrix(
                self.weights[layer_index + 1])
            backpropagate = c_wrapper.multiply_py_matrices(
                transposed_weights, output_error)
            c_wrapper.free_py_matrix(transposed_weights)

            if activation_type == "identity":
                error_signal = backpropagate
            else:
                error_signal = c_wrapper.hadamard_py_matrices(
                    backpropagate, activation_derivative)

            c_wrapper.free_py_matrix(backpropagate)

        # Weight Gradient = Error_Signal * A_{L-1}^T
        transposed_prev = c_wrapper.transpose_py_matrix(
            self.activations[layer_index])

        # Calculate gradients
        weight_grad_temp = c_wrapper.multiply_py_matrices(
            error_signal, transposed_prev)
        weight_grad = c_wrapper.scalar_multiply_py_matrix(
            weight_grad_temp, self.learning_rate)

        bias_grad_temp = c_wrapper.sum_py_matrix_columns(error_signal)
        bias_grad = c_wrapper.scalar_multiply_py_matrix(
            bias_grad_temp, self.learning_rate)

        # Clean up temporary gradient matrices
        c_wrapper.free_py_matrix(weight_grad_temp)
        c_wrapper.free_py_matrix(bias_grad_temp)

        # W_new = W_old - Grad
        new_weights = c_wrapper.subtract_py_matrices(
            self.weights[layer_index], weight_grad)
        new_biases = c_wrapper.subtract_py_matrices(
            self.biases[layer_index], bias_grad)

        # Free the old weights/biases pointers
        c_wrapper.free_py_matrix(self.weights[layer_index])
        c_wrapper.free_py_matrix(self.biases[layer_index])

        # Update to new pointers
        self.weights[layer_index] = new_weights
        self.biases[layer_index] = new_biases

        # Clean up temporary matrices (derivatives, transposes, gradients)
        if activation_derivative is not None:
            c_wrapper.free_py_matrix(activation_derivative)
        c_wrapper.free_py_matrix(transposed_prev)
        c_wrapper.free_py_matrix(weight_grad)
        c_wrapper.free_py_matrix(bias_grad)

        # Free the error matrix from the previous layer
        c_wrapper.free_py_matrix(output_error)
        output_error = error_signal

    c_wrapper.free_py_matrix(output_error)


## `train_model`: Training Loop,

This method executes the main training loop. It first converts and transposes the NumPy input data (`X`, `y`) into the C-wrapper format. For the specified number of epochs, it performs a full forward pass to get predictions, calculates the loss, and executes the backward pass to update the model parameters. The input matrices are freed after training completion.

In [ ]:
def train_model(self, X: np.ndarray, y: np.ndarray, epochs: int):
    X_c = c_wrapper.transpose_py_matrix(c_wrapper.from_numpy(X))
    y_c = c_wrapper.transpose_py_matrix(c_wrapper.from_numpy(y))

    for epoch in range(epochs):
        print(f"Training epoch {epoch}\n")

        y_pred = self.forward_pass(X_c)

        loss = c_wrapper.py_mean_squared_error(y_c, y_pred)
        print(f"This epoch's loss is {loss:.6f}")

        self.backward_pass(y_c, y_pred)

        self._clear_activations()

    c_wrapper.free_py_matrix(X_c)
    c_wrapper.free_py_matrix(y_c)

    print("Training Complete")


## `predict`: Inference

Used for inference on new data. It performs a single **forward pass** on the input data, and importantly, **transposes the final output back** from the internal (features $\times$ samples) format to the external NumPy standard (samples $\times$ features) before returning the results to the user. All temporary C-matrices and activations are freed.

In [ ]:
def predict(self, X: np.ndarray):
    X_c = c_wrapper.transpose_py_matrix(c_wrapper.from_numpy(X))

    y_pred = self.forward_pass(X_c)
    # Transpose the output back to NumPy's (samples, features) format
    y_pred_transposed = c_wrapper.transpose_py_matrix(y_pred)

    prediction = c_wrapper.to_numpy(y_pred_transposed)

    c_wrapper.free_py_matrix(X_c)
    c_wrapper.free_py_matrix(y_pred_transposed)
    self._clear_activations()

    return prediction

To run and evaluate the model, run the below code. A lot of the code to evaluate the model was written by gemini. The reference to the chat is in the appendix. The code is not in this notebook, but a separate library to highlight it's conceptual shift from a theoretical model to a practical implementation.

In [ ]:
!python3 python/run.py